<a href="https://colab.research.google.com/github/BorisLoveDev/agents-of-uniform-difficulty/blob/main/get_answers_table_and_fit10rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import os
import random
np.random.seed(42)
random.seed(42)

In [2]:
from google.colab import drive
# Подключение Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Загрузка и нормализация данных MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

num_classes = 10

# Добавление измерения канала к данным
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Применение one-hot encoding
y_test_ohe = to_categorical(y_test, num_classes)
y_train_ohe = to_categorical(y_train, num_classes)
y_train_ohe.shape

11490434/11490434 [==============================] - 0s 0us/step


(60000, 10)

In [4]:
# Загрузка данных для уровня сложности 0
x_data_level_0 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/x_data_difficult_level_0.npy')
y_data_level_0 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/y_data_difficult_level_0.npy')

# Загрузка данных для уровня сложности 6
x_data_level_6 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/x_data_difficult_level_6.npy')
y_data_level_6 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/y_data_difficult_level_6.npy')


In [5]:
# Функция для деформации изображения, как определено ранее
def deform_image_optimized(image, A, B, M=28, NP=5):
    C = M / (NP + 1.0)
    XN, YN = np.zeros(M), np.zeros(M)
    DX, DY = np.linspace(0, M-1, M), np.linspace(0, M-1, M)

    for j in range(NP):
        TXN = (j + 0.5 - np.random.random()) * C
        TYN = (j + 0.5 - np.random.random()) * C
        TDX = (j + 0.5 - np.random.random()) * C
        TDY = (j + 0.5 - np.random.random()) * C
        AXN = B * (1.0 - 2.0 * np.random.random())
        AYN = B * (1.0 - 2.0 * np.random.random())
        ADX = A * (1.0 - 2.0 * np.random.random())
        ADY = A * (1.0 - 2.0 * np.random.random())
        PXN = (0.1 + 0.9 * np.random.random()) * C
        PYN = (0.1 + 0.9 * np.random.random()) * C
        PDX = (0.1 + 0.9 * np.random.random()) * C
        PDY = (0.1 + 0.9 * np.random.random()) * C

        DX += ADX * np.exp(-((DX - TDX) / PDX)**2)
        DY += ADY * np.exp(-((DY - TDY) / PDY)**2)
        XN += AXN * np.exp(-((DX - TXN) / PXN)**2)
        YN += AYN * np.exp(-((DY - TYN) / PYN)**2)

    deformed_image = np.zeros((M, M))
    for j in range(M):
        for i in range(M):
            x_index = int(DX[i] + XN[j])
            y_index = int(DY[j] + YN[i])
            if 0 <= x_index < M and 0 <= y_index < M:
                deformed_image[j, i] = image[y_index, x_index] if y_index < image.shape[0] and x_index < image.shape[1] else 0

    return deformed_image

In [6]:
deform_params = {
    "1": (0.2, 1.0),
    "2": (0.4, 3.0),
    "3": (1, 5.0),
    "4": (3.5, 9.5),
    "5": (5, 15.0),
}

Получение таблицы ответов для 10 акторов

In [7]:
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization, Activation, LeakyReLU, ELU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow as tf


In [8]:

def create_random_cnn_model(input_shape=(28, 28, 1), num_classes=10):
    model = Sequential()

    conv_layer = random.choice([Conv2D, SeparableConv2D])
    filters = random.randint(16, 64)
    kernel_size = random.choice([(3, 3), (5, 5)])
    activation = random.choice(['relu', 'leaky_relu', 'elu', 'selu', 'swish'])

    model.add(conv_layer(filters=filters, kernel_size=kernel_size, activation=activation, input_shape=input_shape))

    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    num_layers = random.randint(1, 3)
    for _ in range(num_layers):
        conv_layer = random.choice([Conv2D, SeparableConv2D])
        filters = random.randint(16, 128)
        kernel_size = random.choice([(3, 3), (5, 5)])
        activation = random.choice(['relu', 'leaky_relu', 'elu', 'selu', 'swish'])

        if activation == 'selu':
            model.add(conv_layer(filters=filters, kernel_size=kernel_size, activation='selu', padding='same'))
        else:
            model.add(conv_layer(filters=filters, kernel_size=kernel_size, padding='same'))
            if activation == 'leaky_relu':
                model.add(LeakyReLU(alpha=0.01))
            elif activation == 'elu':
                model.add(ELU(alpha=1.0))
            elif activation == 'swish':
                model.add(Activation(tf.nn.swish))
            else:  # relu
                model.add(Activation(activation))

        model.add(BatchNormalization())
        if random.random() < 0.5:
            model.add(MaxPooling2D(pool_size=(2, 2)))
        if random.random() < 0.5:
            model.add(Dropout(rate=random.uniform(0.2, 0.5)))

    model.add(GlobalAveragePooling2D())
    model.add(Dense(units=random.randint(64, 128), activation='relu'))
    if random.random() < 0.5:
        model.add(Dropout(rate=random.uniform(0.2, 0.5)))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Пример использования
model = create_random_cnn_model(input_shape=(28, 28, 1), num_classes=10)
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['accuracy'])


In [9]:
def split_dataset_with_overlap(data, labels, num_chunks, overlap=0.2):
    chunk_size = len(data) // num_chunks
    overlap_size = int(chunk_size * overlap)

    chunks = []
    for i in range(num_chunks):
        start = i * chunk_size - i * overlap_size
        end = start + chunk_size + overlap_size

        if end > len(data):
            end -= len(data)
            chunk_data = np.concatenate((data[start:], data[:end]))
            chunk_labels = np.concatenate((labels[start:], labels[:end]))
        else:
            chunk_data = data[start:end]
            chunk_labels = labels[start:end]

        chunks.append((chunk_data, chunk_labels))

    data_shapes = [chunk[0].shape for chunk in chunks]
    labels_shapes = [chunk[1].shape for chunk in chunks]

    print(f"Формы данных: {data_shapes}")
    print(f"Формы меток: {labels_shapes}")

    return chunks  # Возвращаем список чанков


In [10]:
num_chunks = 10  # Например, разделим на 10 частей
num_classes = 11
chunks = split_dataset_with_overlap(x_train, y_train, num_chunks, overlap=0.2)


Формы данных: [(7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1)]
Формы меток: [(7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,)]


In [11]:
def process_chunks(chunks, num_classes=11):
    processed_chunks = []

    for chunk_data, chunk_labels in chunks:
        # Выбираем случайные A и B для этого чанка
        A = random.uniform(3.5, 12.0 - 0.5)
        A = round(A / 0.5) * 0.5
        B = random.randint(9, 32)

        augmented_images = []
        augmented_labels = []

        # Деформируем и добавляем 1000 случайных изображений как специальный класс 10
        for _ in range(1000):
            random_index = random.randint(0, len(chunk_data) - 1)
            image = chunk_data[random_index].squeeze()
            deformed_image = deform_image_optimized(image, A, B)
            augmented_images.append(deformed_image.reshape(28, 28, 1))
            augmented_labels.append(10)

        # Объединяем исходные данные и метки с дополненными
        chunk_data_augmented = np.concatenate((chunk_data, augmented_images))
        chunk_labels_augmented = np.concatenate((chunk_labels, augmented_labels))

        # Перемешивание дополненных данных и меток
        chunk_data_augmented, chunk_labels_augmented = shuffle(chunk_data_augmented, chunk_labels_augmented, random_state=42)

        # Применение one-hot encoding к дополненным меткам
        chunk_labels_augmented = np.eye(num_classes)[chunk_labels_augmented]

        processed_chunks.append((chunk_data_augmented, chunk_labels_augmented))

    return processed_chunks

In [12]:
def display_special_class_images(processed_chunks):
    for i, (chunk_data, chunk_labels) in enumerate(processed_chunks):
        special_class_indices = np.where(np.argmax(chunk_labels, axis=1) == 10)[0][:4]

        if len(special_class_indices) == 0:
            print(f"В чанке {i} нет данных для специального класса 10.")
            continue

        fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(12, 3))

        for j, index in enumerate(special_class_indices):
            image = chunk_data[index].squeeze()
            axes[j].imshow(image, cmap='gray')
            axes[j].set_title(f"Чанк {i}, Индекс: {index}")
            axes[j].axis('off')

        plt.tight_layout()
        plt.show()


In [13]:
# Пример использования
num_chunks = 10
chunks = split_dataset_with_overlap(x_train, y_train, num_chunks, overlap=0.2)
processed_chunks = process_chunks(chunks)
#display_special_class_images(processed_chunks)

Формы данных: [(7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1)]
Формы меток: [(7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,)]


In [14]:
def print_class_distribution(processed_chunks):
    for i, (chunk_data, chunk_labels) in enumerate(processed_chunks):
        class_counts = np.sum(chunk_labels, axis=0)
        print(f"Чанк {i}:")
        for j, count in enumerate(class_counts):
            print(f"Класс {j}: {count}")
        print()

# Пример использования
processed_chunks = process_chunks(chunks)
print_class_distribution(processed_chunks)

Чанк 0:
Класс 0: 713.0
Класс 1: 809.0
Класс 2: 694.0
Класс 3: 735.0
Класс 4: 734.0
Класс 5: 629.0
Класс 6: 725.0
Класс 7: 770.0
Класс 8: 673.0
Класс 9: 718.0
Класс 10: 1000.0

Чанк 1:
Класс 0: 745.0
Класс 1: 817.0
Класс 2: 704.0
Класс 3: 754.0
Класс 4: 666.0
Класс 5: 629.0
Класс 6: 728.0
Класс 7: 755.0
Класс 8: 682.0
Класс 9: 720.0
Класс 10: 1000.0

Чанк 2:
Класс 0: 724.0
Класс 1: 816.0
Класс 2: 693.0
Класс 3: 759.0
Класс 4: 664.0
Класс 5: 661.0
Класс 6: 689.0
Класс 7: 743.0
Класс 8: 708.0
Класс 9: 743.0
Класс 10: 1000.0

Чанк 3:
Класс 0: 704.0
Класс 1: 844.0
Класс 2: 681.0
Класс 3: 759.0
Класс 4: 693.0
Класс 5: 651.0
Класс 6: 698.0
Класс 7: 720.0
Класс 8: 704.0
Класс 9: 746.0
Класс 10: 1000.0

Чанк 4:
Класс 0: 698.0
Класс 1: 820.0
Класс 2: 726.0
Класс 3: 737.0
Класс 4: 720.0
Класс 5: 674.0
Класс 6: 714.0
Класс 7: 728.0
Класс 8: 680.0
Класс 9: 703.0
Класс 10: 1000.0

Чанк 5:
Класс 0: 703.0
Класс 1: 829.0
Класс 2: 721.0
Класс 3: 703.0
Класс 4: 693.0
Класс 5: 658.0
Класс 6: 725.0
Класс 7

In [15]:
# 1) Создание генератора-аугментатора данных для классификации MNIST
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

In [16]:
# Добавление нулевого класса в тестовые метки
y_test_extended = np.append(y_test_ohe, 0 * np.ones((len(y_test), 1)), axis=1)
y_test_extended[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [17]:
# num_chunks = len(chunks)  # Общее количество чанков
# num_classes = 11

# # Генерация 10 случайных моделей
# models = [create_random_cnn_model(input_shape=x_train.shape[1:], num_classes=num_classes) for _ in range(10)]

# # Компиляция моделей
# for model in models:
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# save_dir = "/content/drive/MyDrive/all_difficulty_data_mnist/weights_100rcnn_normal_train_data"
# os.makedirs(save_dir, exist_ok=True)

# for i, (model, (chunk_data, chunk_labels)) in enumerate(zip(models, chunks)):
#       # Проверка, была ли модель уже обучена (пропускаем первые 5 модели)
#     if i < 5:
#         print(f"Пропуск модели {i+1}, так как она уже обучена.")
#         continue  # Переход к следующей модели

#     # Подготовка валидационного чанка
#     val_chunk_index = (i + 1) % num_chunks
#     val_chunk_data, val_chunk_labels = chunks[val_chunk_index]
#     val_chunk_labels_one_hot = to_categorical(val_chunk_labels, num_classes=num_classes)

#     chunk_labels_one_hot = to_categorical(chunk_labels, num_classes=num_classes)
#     print(f"Обучение модели {i+1}")
#     print(model.summary())
#     # Создание каллбеков
#     model_checkpoint = ModelCheckpoint(os.path.join(save_dir, f"best_model_{i+1}.h5"), save_best_only=True, monitor='val_accuracy', mode='max')
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.001)

#     if i % 2 == 0:  # Для четных индексов используем аугментированные данные
#         train_generator = datagen.flow(chunk_data, chunk_labels_one_hot, batch_size=32)
#         model.fit(train_generator, epochs=100, verbose=1, callbacks=[model_checkpoint, reduce_lr], validation_data=(val_chunk_data, val_chunk_labels_one_hot))
#     else:  # Для нечетных индексов используем неаугментированные данные
#         model.fit(chunk_data, chunk_labels_one_hot, epochs=100, batch_size=32, verbose=1, callbacks=[model_checkpoint, reduce_lr], validation_data=(val_chunk_data, val_chunk_labels_one_hot))

#     # Оценка точности модели на тестовых данных
#     print(f"Тестирование модели {i+1}")
#     loss, accuracy = model.evaluate(x_test, y_test_extended, verbose=0)
#     print(f"Точность модели {i+1} на тестовых данных: {accuracy * 100:.2f}%")

#     # Сохранение модели после обучения
#     model_path = os.path.join(save_dir, f"model_{i+1}.h5")
#     model.save(model_path)
#     print(f"Модель {i+1} сохранена в {model_path}")


In [18]:
import shutil

num_chunks = len(chunks)  # Общее количество чанков
num_classes = 11

# Генерация 10 случайных моделей
models = [create_random_cnn_model(input_shape=x_train.shape[1:], num_classes=num_classes) for _ in range(10)]

# Компиляция моделей
for model in models:
    model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

# Папка на Google Диске для сохранения весов
save_dir = "/content/drive/MyDrive/all_difficulty_data_mnist/weights_100rcnn_normal_train_data"
os.makedirs(save_dir, exist_ok=True)

# Цикл обучения моделей
for i, (model, (chunk_data, chunk_labels)) in enumerate(zip(models, chunks)):
    if i < 3:
        print(f"Пропуск модели {i+1}, так как она уже обучена.")
        continue

    val_chunk_index = (i + 1) % num_chunks
    val_chunk_data, val_chunk_labels = chunks[val_chunk_index]
    val_chunk_labels_one_hot = to_categorical(val_chunk_labels, num_classes=num_classes)

    chunk_labels_one_hot = to_categorical(chunk_labels, num_classes=num_classes)

    print(f"Обучение модели {i+1}")

    # Создание каллбека для сохранения лучших весов локально
    local_model_checkpoint = ModelCheckpoint(f"best_model_{i+1}.h5",
                                             save_best_only=True,
                                             monitor='val_accuracy',
                                             mode='max',
                                             verbose=1)

    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',
                                  factor=0.5,
                                  patience=3,
                                  min_lr=0.0001)

    # Определение, используется ли аугментация данных
    if i % 2 == 0:
        train_generator = datagen.flow(chunk_data, chunk_labels_one_hot, batch_size=32)
        model.fit(train_generator,
                  epochs=100,
                  verbose=1,
                  callbacks=[local_model_checkpoint, reduce_lr],
                  validation_data=(val_chunk_data, val_chunk_labels_one_hot))
    else:
        model.fit(chunk_data,
                  chunk_labels_one_hot,
                  epochs=100,
                  batch_size=32,
                  verbose=1,
                  callbacks=[local_model_checkpoint, reduce_lr],
                  validation_data=(val_chunk_data, val_chunk_labels_one_hot))

    # Тестирование модели
    print(f"Тестирование модели {i+1}")
    loss, accuracy = model.evaluate(x_test, y_test_extended, verbose=0)
    print(f"Точность модели {i+1} на тестовых данных: {accuracy * 100:.2f}%")

    # Сохранение лучших весов на Google Диск
    best_local_path = f"best_model_{i+1}.h5"
    best_drive_path = os.path.join(save_dir, f"best_model_{i+1}.h5")
    if os.path.exists(best_local_path):
        # Используем shutil для копирования файла
        shutil.copy(best_local_path, best_drive_path)
        print(f"Лучшие веса модели {i+1} сохранены на Google Диск: {best_drive_path}")
        # Удаляем локальный файл, если это необходимо
        os.remove(best_local_path)

Пропуск модели 1, так как она уже обучена.
Пропуск модели 2, так как она уже обучена.
Пропуск модели 3, так как она уже обучена.
Обучение модели 4
Epoch 1/100
215/225 [===========================>..] - ETA: 0s - loss: 1.1366 - accuracy: 0.6006
Epoch 1: val_accuracy improved from -inf to 0.36097, saving model to best_model_4.h5
225/225 [==============================] - 9s 10ms/step - loss: 1.1047 - accuracy: 0.6122 - val_loss: 3.6335 - val_accuracy: 0.3610 - lr: 0.0100
Epoch 2/100
 24/225 [==>...........................] - ETA: 0s - loss: 0.4455 - accuracy: 0.8633

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


220/225 [============================>.] - ETA: 0s - loss: 0.4231 - accuracy: 0.8689
Epoch 2: val_accuracy improved from 0.36097 to 0.73014, saving model to best_model_4.h5
225/225 [==============================] - 2s 7ms/step - loss: 0.4240 - accuracy: 0.8690 - val_loss: 0.9537 - val_accuracy: 0.7301 - lr: 0.0100
Epoch 3/100
225/225 [==============================] - ETA: 0s - loss: 0.3094 - accuracy: 0.9039
Epoch 3: val_accuracy improved from 0.73014 to 0.81847, saving model to best_model_4.h5
225/225 [==============================] - 2s 7ms/step - loss: 0.3094 - accuracy: 0.9039 - val_loss: 0.5834 - val_accuracy: 0.8185 - lr: 0.0100
Epoch 4/100
215/225 [===========================>..] - ETA: 0s - loss: 0.2753 - accuracy: 0.9102
Epoch 4: val_accuracy improved from 0.81847 to 0.83319, saving model to best_model_4.h5
225/225 [==============================] - 2s 8ms/step - loss: 0.2738 - accuracy: 0.9100 - val_loss: 0.6381 - val_accuracy: 0.8332 - lr: 0.0100
Epoch 5/100
220/225 [====